In [1]:
#export
import csv
import numpy as np
import ast
from datetime import datetime
from math import log, floor, ceil
import random
import numpy as np

In [2]:
#export
class Utility(object):
    
    # This method computes entropy for information gain
    def entropy(self, class_y):
        # Input:
        # class_y: list of class labels (0's and 1's)

        # ToDo: Compute the entropy for a list of classes
        #
        # Example:
        #    entropy([0,0,0,1,1,1,1,1,1]) = 0.918 (rounded to three decimal places)

        entropy = 0
        ## calculate probability of each class (1/0)
        pr_0 = len([y for y in class_y if y==0])/len(class_y)    
        pr_1 = len([y for y in class_y if y==1])/len(class_y)
        
        ## if prob is 0 for either class, return untreated entropy
        if pr_0 == 0 or pr_1 == 0:
            return entropy

        ## compute entropy
        entropy = - pr_0 * np.log2(pr_0) - pr_1 * np.log2(pr_1)

        return entropy


    def partition_classes(self, X, y, split_attribute, split_val):
        # Inputs:
        #   X               : data containing all attributes
        #   y               : labels
        #   split_attribute : column index of the attribute to split on
        #   split_val       : a numerical value to divide the split_attribute

 

        # ToDo: Partition the data(X) and labels(y) based on the split value - BINARY SPLIT.
        
        #   Splitting the data X into two lists(X_left and X_right) where the first list has all
        #   the rows where the split attribute is less than or equal to the split value, and the 
        #   second list has all the rows where the split attribute is greater than the split 
        #   value. Also creating two lists(y_left and y_right) with the corresponding y labels.


        X_left = []
        X_right = []

        y_left = []
        y_right = []
        
        for i in range(len(X)):
            if X[i][split_attribute] <= split_val:
                X_left.append(X[i])
                y_left.append(y[i])
            else:
                X_right.append(X[i])
                y_right.append(y[i])

        return (X_left, X_right, y_left, y_right)


    def information_gain(self, previous_y, current_y):
        # Inputs:
        #   previous_y: the distribution of original labels (0's and 1's)
        #   current_y:  the distribution of labels after splitting based on a particular
        #               split attribute and split value

        # ToDo: Compute and return the information gain from partitioning the previous_y labels
        # into the current_y labels.
        # Reference: http://www.cs.cmu.edu/afs/cs.cmu.edu/academic/class/15381-s06/www/DTs.pdf

        info_gain = 0
        # getting previous entropy
        previous_entropy = self.entropy(previous_y)
        ## getting a list of entropy values for splitting
        after_entropy =[self.entropy(y) if len(y)>0 else 0 for y in current_y]
        ## getting probabilities for split
        probs = [len(y)/len(previous_y) for y in current_y]
        ## getting the entropy after split
        after_entropy = sum([after_entropy[i] * probs[i] for i in range(len(probs))])
        ## getting the info gain
        info_gain = previous_entropy - after_entropy

        #############################################
        return info_gain


    def best_split(self, X, y):
        # Inputs:
        #   X       : Data containing all attributes
        #   y       : labels
        #   ToDo    : For each node find the best split criteria and return the split attribute, 
        #             spliting value along with  X_left, X_right, y_left, y_right 
        
        split_attribute = 0
        split_val = 0
        X_left, X_right, y_left, y_right = [], [], [], []
        
        num_attrs = len(X[0])
        chosen_atrrs = np.random.choice(num_attrs,num_attrs//2, replace=False)
        max = -float("Inf")
        return_dict = {}

        ## find the split that maximizes info gain via iteration
        for attr in chosen_atrrs:
            for row in range(len(X)):
                X_left_tmp, X_right_tmp, y_left_tmp, y_right_tmp = self.partition_classes(X, y, attr, X[row][attr])
                current_y = [y_left_tmp, y_right_tmp]
                info_gain = self.information_gain(y, current_y)

                if max < info_gain:
                    X_left, X_right, y_left, y_right = X_left_tmp, X_right_tmp, y_left_tmp, y_right_tmp
                    split_attribute = attr
                    split_value = X[row][attr]
                    max = info_gain
        
        #insert outputs to a dictionary                    
        return_dict['X_left'] = X_left
        return_dict['X_right'] = X_right
        return_dict['y_left'] = y_left
        return_dict['y_right'] = y_right
        return_dict['split_attribute'] = split_attribute
        return_dict['split_value'] = split_value
        return_dict['info_gain'] = info_gain
        
        return return_dict
        


### Define the classes 'DecisionTree' and 'RandomForest'

In [3]:
#export
class DecisionTree(object):
    def __init__(self, max_depth):
        # Initializing the tree as an empty dictionary or list, as preferred
        #self.tree = {}
        self.tree = []
        self.max_depth = max_depth
        
    	
    def learn(self, X, y, depth=0):
        # ToDo: Train the decision tree (self.tree) using the the sample X and labels y
       
        util = Utility()
        entropy_y = util.entropy(y)
        if entropy_y == 0:
            return [-1, y[0], None, None]

        gain_best = 0
        gain_best_idx = -1
        value_min=[]
        X_left_min=[]
        X_right_min=[]
        y_left_min=[]
        y_right_min=[]
        
        length = len(X[0])

        for curr_idx in range(0, length):      
            local_val=np.mean([X[i][curr_idx] for i in range(len(X))])
            X_left, X_right, y_left, y_right=util.partition_classes(X, y, curr_idx, local_val)
            gain = util.information_gain(y, [y_left, y_right])

            if gain > gain_best:
                gain_best = gain
                gain_best_idx = curr_idx
                value_min = local_val
                X_left_min, X_right_min, y_left_min, y_right_min= X_left, X_right, y_left, y_right
        
        if gain_best <= 0:
            return [-1, np.argmax(np.bincount(y)),None, None]

        left=self.learn(X_left_min,y_left_min)
        right=self.learn(X_right_min, y_right_min)

        self.tree=[gain_best_idx, value_min, left, right]

        return self.tree

    def classify(self, record):
        # ToDo: classify the record using self.tree and return the predicted label
        def classify_additional(record, tree):
            idx, val, left, right = tree
            if idx == -1:
                return val
            else:
                if record[idx] <= val:
                    return classify_additional(record, left)
                else:
                    return classify_additional(record, right)
        return classify_additional(record, self.tree)

In [4]:
#export

# 1. X is assumed to be a matrix with n rows and d columns where n is the number of total records and d is the number of features of each record. 
# 2. y is assumed to be a vector of labels of length n.
# 3. XX is similar to X, except that XX also contains the data label for each record.

class RandomForest(object):
    num_trees = 0
    decision_trees = []

    # bootstrapping datasets for trees
    # bootstraps_datasets is a list of lists, where each list in bootstraps_datasets is a bootstrapped dataset.
    bootstraps_datasets = []

    # the true class labels, corresponding to records in the bootstrapping datasets
    # bootstraps_labels is a list of lists, where the 'i'th list contains the labels corresponding to records in
    # the 'i'th bootstrapped dataset.
    bootstraps_labels = []

    def __init__(self, num_trees):
        # Initialization done here
        self.num_trees = num_trees
        self.decision_trees = [DecisionTree(max_depth=10) for i in range(num_trees)]
        self.bootstraps_datasets = []
        self.bootstraps_labels = []
        
    def _bootstrapping(self, XX, n):
        # Reference: https://en.wikipedia.org/wiki/Bootstrapping_(statistics)
        #
        # ToDo: Create a sample dataset of size n by sampling with replacement
        #       from the original dataset XX.
        # Note that you would also need to record the corresponding class labels
        # for the sampled records for training purposes.

        sample = [] # sampled dataset
        labels = []  # class labels for the sampled records
        
        for i in range(n):
            idx = np.random.randint(len(XX))
            row = XX[idx]
            sample.append(row[:-1]) #everything except last element
            labels.append(row[-1]) #only last element

        return (sample, labels)

    def bootstrapping(self, XX):
        # Initializing the bootstap datasets for each tree
        for i in range(self.num_trees):
            data_sample, data_label = self._bootstrapping(XX, len(XX))
            self.bootstraps_datasets.append(data_sample)
            self.bootstraps_labels.append(data_label)

    def fitting(self):
        # ToDo: Train `num_trees` decision trees using the bootstraps datasets
        # and labels by calling the learn function from your DecisionTree class.
        for i in range(self.num_trees):
            self.decision_trees[i].learn(self.bootstraps_datasets[i], self.bootstraps_labels[i])
        return(self.decision_trees[0].tree)

    def voting(self, X):
        y = []

        for record in X:
            #   1. Find the set of trees that consider the record as an
            #      out-of-bag sample.
            #   2. Predict the label using each of the above found trees.
            #   3. Use majority vote to find the final label for this recod.
            votes = []
            
            for i in range(len(self.bootstraps_datasets)):
                dataset = self.bootstraps_datasets[i]
                
                if record not in dataset:
                    OOB_tree = self.decision_trees[i]
                    effective_vote = OOB_tree.classify(record)
                    votes.append(effective_vote)

            counts = np.bincount(votes)

            if len(counts) == 0:
                #  Handle the case where the record is not an out-of-bag sample for any of the trees.
                i = self.bootstraps_datasets[0].index(record)
                y = np.append(y, self.bootstraps_labels[0][i])
            else:
                y = np.append(y, np.argmax(counts))
                               
        return y

In [5]:
#export

# ToDo: Determine the forest size according to your implementation. 
def get_forest_size():
    forest_size = 10
    return forest_size

# ToDo: Determine random seed to set for reproducibility
def get_random_seed():
    random_seed = 0
    return random_seed

In [6]:
def run():
    np.random.seed(get_random_seed())
    # start time 
    start = datetime.now()
    X = list()
    y = list()
    XX = list()  # Contains data features and data labels
    numerical_cols = set([i for i in range(0, 9)])  # indices of numeric attributes (columns)

    # Loading data set
    print("reading the data")
    with open("pima-indians-diabetes.csv") as f:
        next(f, None)
        for line in csv.reader(f, delimiter=","):
            xline = []
            for i in range(len(line)):
                if i in numerical_cols:
                    xline.append(ast.literal_eval(line[i]))
                else:
                    xline.append(line[i])

            X.append(xline[:-1])
            y.append(xline[-1])
            XX.append(xline[:])

    # Initializing a random forest.
    randomForest = RandomForest(get_forest_size())


    # Creating the bootstrapping datasets
    print("creating the bootstrap datasets")
    randomForest.bootstrapping(XX)

    # Building trees in the forest
    print("fitting the forest")
    randomForest.fitting()

    # Calculating an unbiased error estimation of the random forest
    # based on out-of-bag (OOB) error estimate.
    y_predicted = randomForest.voting(X)

    # Comparing predicted and true labels
    results = [prediction == truth for prediction, truth in zip(y_predicted, y)]

    # Accuracy
    accuracy = float(results.count(True)) / float(len(results))

    print("accuracy: %.4f" % accuracy)
    print("OOB estimate: %.4f" % (1 - accuracy))

    # end time
    print("Execution time: " + str(datetime.now() - start))

In [7]:
 %run helpers/notebook2script random-forest

Converted RF-demo.ipynb to random-forest\random-forest.py


In [8]:
# Call the run() function to test your implementation
# Use this cell and any cells below for additional testing
run()

reading the data
creating the bootstrap datasets
fitting the forest
accuracy: 0.7135
OOB estimate: 0.2865
Execution time: 0:00:00.731417
